In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy  as np

from lazypredict.Supervised import LazyRegressor 
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

from  sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold

from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score , mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [ ]:
with open("databangladeshi_users.json") as f :
    jsn = json.load(f)


In [325]:
flat_data = []

for user_data in jsn :
    flat_data.append({
        "purchases" : sum(user_data["purchases"]),
        "items" : len(user_data["purchases"]),
        "average" : sum(user_data["purchases"])/len(user_data["purchases"]),
        "location" : user_data["location"],
        "lastlogin" : user_data["last_login"],
        "loyalty" : user_data["loyalty"]
    })

df = pd.DataFrame(flat_data)
df.head()

,purchases,items,average,location,lastlogin,loyalty
0,4931,4,1232.75,Barisal,2025-04-25,Bronze
1,1137,2,568.50,Chittagong,2025-04-30,Bronze
2,2483,2,1241.50,Rajshahi,2025-05-06,Bronze
3,1986,3,662.00,Mymensingh,2025-03-24,Bronze
4,4862,5,972.40,Khulna,2025-05-20,Bronze


Cleaning and encoding

In [327]:
df["lastlogin"] = pd.to_datetime(df["lastlogin"])

df["lastseen"] = (datetime.today() - df["lastlogin"]).dt.days


df = df.drop(columns= "lastlogin")

df.head()

,purchases,items,average,location,loyalty,lastseen
0,4931,4,1232.75,Barisal,Bronze,30
1,1137,2,568.50,Chittagong,Bronze,25
2,2483,2,1241.50,Rajshahi,Bronze,19
3,1986,3,662.00,Mymensingh,Bronze,62
4,4862,5,972.40,Khulna,Bronze,5


In [328]:
le_loc = LabelEncoder()
le_loyl = LabelEncoder()

df["location"] = le_loc.fit_transform(df["location"])

df["loyalty"] = le_loyl.fit_transform(df["loyalty"])

df.head()

,purchases,items,average,location,loyalty,lastseen
0,4931,4,1232.75,0,0,30
1,1137,2,568.50,1,0,25
2,2483,2,1241.50,5,0,19
3,1986,3,662.00,4,0,62
4,4862,5,972.40,3,0,5


splitting

In [329]:
x = df.drop(["purchases"], axis=1)  
y = df["purchases"]               

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


lazypredict regression

In [330]:

reg = LazyRegressor( ignore_warnings=True, random_state=42)

models, predictions = reg.fit(x_train, x_test, y_train, y_test)
models

  0%|          | 0/42 [00:00<?, ?it/s]

C:\Users\VICTUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  "Stochastic Optimizer: Maximum iterations (%d) "
C:\Users\VICTUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,1.00,1.00,31.40,0.28
RandomForestRegressor,1.00,1.00,77.17,0.37
BaggingRegressor,1.00,1.00,81.24,0.07
XGBRegressor,1.00,1.00,100.70,0.17
HistGradientBoostingRegressor,1.00,1.00,104.29,0.52
GradientBoostingRegressor,1.00,1.00,107.19,0.20
DecisionTreeRegressor,1.00,1.00,107.73,0.02
ExtraTreeRegressor,1.00,1.00,114.93,0.02
LGBMRegressor,1.00,1.00,118.72,0.08


tuning

In [ ]:
enet = ElasticNet(max_iter=10000)
param = {
    "alpha": [0.001, 0.01, 0.1, 0.5, 1, 10],
    "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9],
    "random_state": list(range(1, 100)),
    "selection": ['cyclic', 'random']
}

In [341]:
grd_srch = GridSearchCV(enet,param,cv=5,scoring="r2")
grd_srch.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=ElasticNet(max_iter=10000),
             param_grid={'alpha': [0.001, 0.01, 0.1, 0.5, 1, 10],
                         'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
                         'random_state': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20, 21,
                                          22, 23, 24, 25, 26, 27, 28, 29, 30, ...],
                         'selection': ['cyclic', 'random']},
             scoring='r2')

In [342]:
print("Best Parameters:", grd_srch.best_params_)
print("Best R² Score on CV:", grd_srch.best_score_)


Best Parameters: {'alpha': 0.1, 'l1_ratio': 0.7, 'random_state': 20, 'selection': 'random'}
Best R² Score on CV: 0.9157571396217558


Model

In [354]:

enet = ElasticNet(alpha= 0.1, l1_ratio= 0.7,  random_state=20, selection= 'random')
enet.fit(x_train,y_train)

y_train_pred = enet.predict(x_train )
y_test_pred = enet.predict(x_test)

Evaluate

In [355]:
MSE = mean_squared_error(y_test,y_test_pred)
RMSE = np.sqrt(MSE)


print("Train R²:", r2_score(y_train, y_train_pred))
print("Test R²:", r2_score(y_test, y_test_pred))
print("MSE : ",MSE )
print("RMSE:", RMSE)

Train R²: 0.9230635494586872
Test R²: 0.9179023226840927
MSE :  304626.2938027399
RMSE: 551.9296094636886


validation

In [357]:

kf = KFold(n_splits=5, shuffle=True, random_state=42)

r2_scores = cross_val_score(enet, x, y, cv=5, scoring='r2')

print("Cross-validated R² scores:", r2_scores)
print("Average R² score:", np.mean(r2_scores))
print("Standard Deviation of R²:", np.std(r2_scores))


neg_mse_scores = cross_val_score(enet, x, y, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-neg_mse_scores)
print("CV RMSE Scores:", rmse_scores)
print("Mean CV RMSE:", np.mean(rmse_scores))

Cross-validated R² scores: [0.90203158 0.93422587 0.91231293 0.92750835 0.90921282]
Average R² score: 0.9170583085249626
Standard Deviation of R²: 0.011948234831537844
CV RMSE Scores: [574.16036681 532.43878396 574.06291363 511.36305334 557.56810065]
Mean CV RMSE: 549.9186436780792
